In [1]:
import pandas as pd

In [2]:
targets = ["CHEMBL226", "CHEMBL240", "CHEMBL251"]
cols = [
        "target_chembl_id",
        "smiles",
        "pchembl_value",
        "comment",
        "standard_type",
        "standard_relation",
        "document_year",
       ]
px_placeholder = 3.99
temporal_split_year = 2015

In [3]:
df = pd.read_csv("../data/raw/ligand_raw.tsv", sep="\t")

In [5]:
df.columns = df.columns.str.lower()
df.dropna(subset=["smiles"], inplace=True)

In [6]:
df = df[df["target_chembl_id"].isin(targets)]

In [7]:
s_year = df.groupby("smiles")["document_year"].min().dropna()
s_year = s_year.astype("Int16")

In [8]:
idx_test = s_year[s_year > 2015].index

In [11]:
df = df[cols].set_index(['target_chembl_id', 'smiles'])

In [12]:
numery = df['pchembl_value'].groupby(['target_chembl_id', 'smiles']).mean().dropna()

In [15]:
comments = df[(df['comment'].str.contains('Not Active') == True)]

In [19]:
inhibits = df[(df['standard_type'] == 'Inhibition') & df['standard_relation'].isin(['<', '<='])]

In [21]:
relations = df[df['standard_type'].isin(['EC50', 'IC50', 'Kd', 'Ki']) & df['standard_relation'].isin(['>', '>='])]

In [25]:
negatives = pd.concat([comments, inhibits, relations], axis=0)

In [29]:
negatives_clean = negatives[~negatives.index.isin(numery.index)]

In [31]:
negatives_clean.loc[:, 'pchembl_value'] = px_placeholder

/tmp/ipykernel_122455/4128785534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negatives_clean.loc[:, 'pchembl_value'] = px_placeholder


In [33]:
 negatives_clean = negatives_clean['pchembl_value'].groupby(['target_chembl_id', 'smiles']).first()

In [34]:
negatives_clean

target_chembl_id  smiles                                                      
CHEMBL226         Br.CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)C(O)c1ccccc1)C2               3.99
                  Brc1c(NC2=NCCN2)ccc2nccnc12                                     3.99
                  Brc1cccc(Nc2nc3c(N4CCCC4)ncnc3s2)c1                             3.99
                  Brc1ccccc1                                                      3.99
                  C#CC1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3CC[C@@]21CC    3.99
                                                                                  ... 
CHEMBL251         c1cncc(Nc2ncc(-c3ccncn3)c(-c3ccco3)n2)c1                        3.99
                  c1coc(-c2cn3ccccc3n2)c1                                         3.99
                  c1coc(-c2nc(Nc3cnccn3)ncc2-c2ccncn2)c1                          3.99
                  c1coc(-c2nc(Nc3cncnc3)ncc2-c2ccncn2)c1                          3.99
                  c1nc2c(NC3CCCCC3)nc(Nc3ccc(N4CCOC

In [39]:
dfx = pd.concat([numery, negatives_clean])

In [45]:
dfx = dfx.unstack('target_chembl_id')

In [50]:
idx_test_p = list(set(dfx.index).intersection(idx_test))

In [51]:
df_test = dfx.loc[idx_test_p]

In [53]:
df_train = dfx.drop(index=df_test.index)

In [92]:
df_train['CHEMBL226'].dropna().index.values

array(['Br.CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)C(O)c1ccccc1)C2',
       'Brc1c(NC2=NCCN2)ccc2nccnc12', 'Brc1ccccc1', ...,
       'c1csc(-c2nc(C3CC3)nc3ccsc23)n1',
       'c1nc2c(NC3CCCCC3)nc(Nc3ccc(N4CCOCC4)cc3)nc2[nH]1',
       'c1nc2nc(Nc3ccc(N4CCOCC4)cc3)nc(NC3CC4CCC3C4)c2[nH]1'],
      dtype=object)

In [109]:
import numpy as np

np.empty(0)

array([], dtype=float64)

In [118]:
from dataclasses import dataclass
from typing import Dict, List, Any
from rxitect.chem.utils import calc_fp
from dataclasses import field

@dataclass
class QSARDataset:
    dataset: pd.DataFrame
    idx_test_temporal_split: pd.Index
    params_used: Dict[str, Any]
    df_test: pd.DataFrame = field(init=False)
    df_train: pd.DataFrame = field(init=False)
    _X_train: Dict[str, Any] = field(init=False)
    _X_test: Dict[str, Any] = field(init=False)
    _y_train: Dict[str, Any] = field(init=False)
    _y_test: Dict[str, Any] = field(init=False)
    
    def __post_init__(self) -> None:
        """Initializes the train and test data based on a temporal split in the data to be used for QSAR model fitting."""
        self.df_test = self.dataset.loc[self.idx_test_temporal_split]
        self.df_train = self.dataset.drop(self.df_test.index)
        self._X_train = {k: np.array([]) for k in self.params_used["targets"]}
        self._X_test = {k: np.array([]) for k in self.params_used["targets"]}
        self._y_train = {k: np.array([]) for k in self.params_used["targets"]}
        self._y_test = {k: np.array([]) for k in self.params_used["targets"]}
        
    def X_train(self, target_chembl_id: str) -> np.ndarray:
        """Lazily evaluates the train data points for a given target ChEMBL ID
        
        Args:
            target_chembl_id:
        
        Returns:
            An array containing the fingerprints of all train data points for the given target ChEMBL ID
        """
        if not self._X_train[target_chembl_id].size:
            data = self.df_train[target_chembl_id].dropna().index
            self._X_train[target_chembl_id] = calc_fp(data)
        return self._X_train[target_chembl_id]
    
    def X_test(self, target_chembl_id: str) -> np.ndarray:
        """Lazily evaluates the train data points for a given target ChEMBL ID
        
        Args:
            target_chembl_id:
        
        Returns:
            An array containing the fingerprints of all test data points for the given target ChEMBL ID
        """
        if not self._X_test[target_chembl_id].size:
            data = self.df_test[target_chembl_id].dropna().index
            self._X_test[target_chembl_id] = calc_fp(data)
        return self._X_test[target_chembl_id]
    
    def y_train(self, target_chembl_id: str) -> np.ndarray:
        """Lazily evaluates the train data points for a given target ChEMBL ID
        
        Args:
            target_chembl_id:
        
        Returns:
            An array containing the pChEMBL value of all train data points for the given target ChEMBL ID
        """
        if not self._y_train[target_chembl_id].size:
            data = self.df_train[target_chembl_id].dropna().values
            self._y_train[target_chembl_id] = data
        return self._y_train[target_chembl_id]
    
    def y_test(self, target_chembl_id: str) -> np.ndarray:
        """Lazily evaluates the train data points for a given target ChEMBL ID
        
        Args:
            target_chembl_id:
        
        Returns:
            An array containing the pChEMBL value of all test data points for the given target ChEMBL ID
        """
        if not self._y_test[target_chembl_id].size:
            data = self.df_test[target_chembl_id].dropna().values
            self._y_test[target_chembl_id] = data
        return self._y_test[target_chembl_id]

In [146]:
def construct_qsar_dataset(raw_data_path: str, out_data_path: str, targets: List[str], cols: List[str],
                           px_placeholder: float = 3.99, temporal_split_year: int = 2015,
                           negative_samples: bool = True) -> QSARDataset:
    """Method that constructs a dataset from ChEMBL data to train QSAR regression models on,
    using a temporal split to create a hold-out test dataset for evaluation.
    
    Args:
        raw_data_path:
        out_data_path:
        targets:
        cols:
        px_placeholder:
        temporal_split_year:
        negative_samples:
        
    Returns:
        A QSARDataset object
    """
    # Load and standardize raw data
    df = pd.read_csv(raw_data_path, sep='\t')
    df.columns = df.columns.str.lower()
    df.dropna(subset=['smiles'], inplace=True)
    
    # Filter data to only contain relevant targets
    df = df[df['target_chembl_id'].isin(targets)]
    
    # Create temporal split for hold-out test set creation downstream
    s_year = df.groupby("smiles")["document_year"].min().dropna()
    s_year = s_year.astype("Int16")
    idx_test = s_year[s_year > 2015].index
    
    # Re-index data to divide SMILES per target
    df = df[cols].set_index(['target_chembl_id', 'smiles'])
    
    # Process positive examples from data, taking the mean of duplicates and removing missing entries
    pos_samples = df['pchembl_value'].groupby(['target_chembl_id', 'smiles']).mean().dropna()
    
    df_processed = pos_samples
    if negative_samples:
        # Process negative examples from data, setting their default pChEMBL values to some threshold (default 3.99)
        # Looks for where inhibition or no activity are implied
        comments = df[(df['comment'].str.contains('Not Active') == True)]
        inhibitions = df[(df['standard_type'] == 'Inhibition') & df['standard_relation'].isin(['<', '<='])]
        relations = df[df['standard_type'].isin(['EC50', 'IC50', 'Kd', 'Ki']) & df['standard_relation'].isin(['>', '>='])]
        neg_samples = pd.concat([comments, inhibitions, relations])
        # Ensure only true negative samples remain in the negative sample set
        neg_samples = neg_samples[~neg_samples.index.isin(pos_samples.index)]
        neg_samples['pchembl_value'] = px_placeholder
        neg_samples = neg_samples['pchembl_value'].groupby(['target_chembl_id', 'smiles']).first()  # Regroup indices
        df_processed = pd.concat([pos_samples, neg_samples])
        
    df_processed = df_processed.unstack('target_chembl_id')
    idx_test = list(set(df_processed.index).intersection(idx_test))

    # df_test = df_processed.loc[idx_test]
    # df_train = df_processed.drop(index=df_test.index)
    
    qsar_dataset = QSARDataset(dataset=df_processed,
                               idx_test_temporal_split=idx_test,
                               params_used={
                                   "targets": targets,
                                   "px_placeholder": px_placeholder,
                                   "temporal_split_year": temporal_split_year,
                                   "negative_samples": negative_samples,
                               })

    return qsar_dataset, df_test, df_train


In [147]:
data, test, train = construct_qsar_dataset(raw_data_path="../data/raw/ligand_raw.tsv",
                       out_data_path="",
                       targets=targets,
                       cols=cols,
                       px_placeholder=px_placeholder,
                       temporal_split_year=temporal_split_year,
                       negative_samples=True)

In [148]:
train

target_chembl_id,CHEMBL226,CHEMBL240,CHEMBL251
smiles,,,
Br.CCn1c(O)c(N=NC2=NC(=O)CS2)c2ccccc21,NaN,5.32,NaN
Br.CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)C(O)c1ccccc1)C2,3.99,3.99,3.99
Brc1c(NC2=NCCN2)ccc2nccnc12,3.99,3.99,3.99
Brc1cccc2nc(C#Cc3ccccn3)ccc12,NaN,4.64,NaN
Brc1ccccc1,3.99,3.99,3.99
...,...,...,...
c1nc2c(NC3CCCCC3)nc(Nc3ccc(N4CCOCC4)cc3)nc2[nH]1,3.99,NaN,3.99
c1nc2nc(Nc3ccc(N4CCOCC4)cc3)nc(NC3CC4CCC3C4)c2[nH]1,4.47,NaN,4.41
c1ncc(-c2cc3sc(N4CCC(N5CCCCC5)CC4)nc3cn2)cn1,NaN,5.19,NaN


In [145]:
chembl226_fps = data.X_train("CHEMBL226")

Calculating physico-chemical properties: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 340.41it/s]


In [131]:
data.dataset["CHEMBL226"].dropna()

smiles
Br.CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)C(O)c1ccccc1)C2               3.99
Brc1c(NC2=NCCN2)ccc2nccnc12                                     3.99
Brc1cccc(Nc2nc3c(N4CCCC4)ncnc3s2)c1                             3.99
Brc1ccccc1                                                      3.99
C#CC1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3CC[C@@]21CC    3.99
                                                                ... 
c1coc(-c2nc3nc(NC4CCCCC4)nc(NC4CCCCC4)n3n2)c1                   6.51
c1coc(CNc2nc3ccccc3n3nc(-c4ccco4)nc23)c1                        7.48
c1csc(-c2nc(C3CC3)nc3ccsc23)n1                                  6.59
c1nc2c(NC3CCCCC3)nc(Nc3ccc(N4CCOCC4)cc3)nc2[nH]1                3.99
c1nc2nc(Nc3ccc(N4CCOCC4)cc3)nc(NC3CC4CCC3C4)c2[nH]1             4.47
Name: CHEMBL226, Length: 5952, dtype: float64

In [133]:
data.df_test

target_chembl_id,CHEMBL226,CHEMBL240,CHEMBL251
smiles,,,
CC(CCc1ccccc1)Nc1nc2nn(C)cc2c2nc(-c3ccco3)nn12,6.88,NaN,6.83
CCCCC#Cc1nc(N)c2ncn([C@@H]3O[C@H](CSCCC)[C@@H](O)[C@H]3O)c2n1,NaN,NaN,6.10
C=CCn1c(=O)c2[nH]c(Cc3cccs3)nc2n(Cc2ccco2)c1=O,NaN,NaN,5.84
Cc1cc(-c2ccc3c(c2)CCN(CCCSc2nnc(-c4ccccc4Cl)n2C)CC3)no1,NaN,7.20,NaN
Cc1cccnc1CNC(=O)c1cc(-c2ccccc2F)nc(N)n1,7.42,NaN,7.85
...,...,...,...
COc1ccc(-c2nc(COc3cccc(CN(CC(=O)O)C(=O)Oc4ccc(C)cc4)c3)c(C)o2)cc1,NaN,5.07,NaN
CCOC(=O)[C@@]12C[C@@H]1[C@@H](n1cnc3c(NC(C4CC4)C4CC4)nc(Cl)nc31)[C@H](O)[C@@H]2O,6.44,NaN,5.80
CCCn1c(=O)c2nc(-c3cnn(Cc4nc(-c5ccccc5OC)no4)c3)[nH]c2n(CCC)c1=O,5.72,NaN,5.80
